In [1]:
from bs4 import BeautifulSoup as bs              
import urllib.request as req                     
import time
from datetime import date, timedelta
import gc
import re
from selenium import webdriver
import sqlite3

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("start-fullscreen")
prefs = {"profile.managed_default_content_settings.images":2}
options.add_experimental_option("prefs",prefs)

In [6]:
sections_url = {
    #comm
    '통신방송' : ['http://www.etnews.com/news/section.html?id1=03&page=','comm'],
    
    #swgm
    'SW게임성장' : ['http://www.etnews.com/news/section.html?id1=04&page=','swgm'],
    
    #cpnt
    '소재부품' : ['http://www.etnews.com/news/section.html?id1=06&page=','cpnt'],
    
    #elec
    '전자자동차유통' : ['http://www.etnews.com/news/section.html?id1=60&page=','elec'],
    
    #finc
    '경제금융' : ['http://www.etnews.com/news/section.html?id1=02&page=','finc'],
    
    #inds
    '산업과학정책' : ['http://www.etnews.com/news/section.html?id1=20&page=','inds']
    } 

In [3]:
def ready_to_scrap_page_main(url):
    _ = req.Request(url)
    HTML = req.urlopen(_)
    page = bs(HTML.read(), 'html.parser')
    articles = dict()
    for article in page.find('ul',{'class':'list_news'}).findAll('dl',{'class':'clearfix'}):
        if re.compile(u'headline clearfix').search(str(article)) != None:
            continue
        articles[article.find('dt').find('a')['href']] = article.find('dt').find('a').get_text()

    return articles
    

def start_to_scrap_article(url_set):
    driver.get(url_set)
    HTML = driver.page_source 
    page = bs(HTML, 'html.parser')
    article = page.find('section',{'class':'article_body'}).find('p').get_text()
    return article

def error_manager(x):
    #x = x.replace(',','').replace('“','`').replace("”",'`').replace("'",'`').replace('(','').replace(')','').replace('?','').replace('·','')
    x = x.replace('“','`').replace("”",'`').replace("'",'`').replace('(','').replace(')','').replace('?','').replace('·',' ')
    return x

def upload_data_to_db(data, table):
    conn = sqlite3.connect('ETNEWS.db')
    cur = conn.cursor()
    for d in data:
        d = list(d.values())
        sql = "insert into {} values ('{}','{}','{}','{}')".format(table, 
                                                           error_manager(d[0]),
                                                           error_manager(d[1]),
                                                           error_manager(d[2]),
                                                           error_manager(d[3]))
        cur.execute(sql)
        conn.commit()
    conn.close()
    
def check_date_last_update():
    conn = sqlite3.connect('ETNEWS.db')
    cur = conn.cursor()
    sql = "select max(*) from last_update_date"
    cur.execute(sql)
    final_update = cur.fetchone()
    return final_update

def record_date_last_update(date):
    conn = sqlite3.connect('ETNEWS.db')
    cur = conn.cursor()
    sql = "insert into last_update_date values({})".format(date)
    cur.execute(sql)
    conn.commit()
    conn.close()

In [8]:
#2주 전까지만 신문 열람 가능
yesterday = date.today() - timedelta(1)
today_date = yesterday.strftime('%Y%m%d')
#final_update = check_date_last_update()
final_update = '20180715'

driver = webdriver.Chrome("C:/chromedriver.exe", chrome_options=options)

for section in sections_url:
    print(section," 탐색 시작.")
    page_number = 1
    overdate_flag = 0
    save_matrix = []
    while True:
        time.sleep(1)
        for article in ready_to_scrap_page_main(sections_url[section][0]+str(page_number)).items():
            gc.collect()
            if re.compile('20[0-9]{6}').search(article[0]).group() == time.strftime('%Y%m%d'):
                continue
            elif re.compile('20[0-9]{6}').search(article[0]).group() == final_update:
                overdate_flag = 1
                continue
            else :
                save_matrix.append({
                    'title': article[1], #기사제목
                    'content':start_to_scrap_article(article[0]), #기사내용
                    'date':re.compile('20[0-9]{6}').search(article[0]).group(),
                    'url_id':re.compile('20[0-9]{12}').search(article[0]).group()
                })
                
        page_number += 1 #한페이지의 기사 스크랩이 끝나서 다음페이지로 넘어가기 위함.
        if overdate_flag == 1: break
    
    print("============= DB에 저장합니다. ============")
    upload_data_to_db(save_matrix, sections_url[section][1])
    print("============= 현재 섹션을 종료합니다. ============")
    
driver.close()

record_date_last_update(today_date)

산업과학정책  탐색 시작.
============= DB에 저장합니다. ============
============= 현재 섹션을 종료합니다. ============


In [7]:
driver.close()